1. Elegir una tarea (diferente de “text classification”). En el siguiente enlace, en la 
sección de “Natural Language Processing” se pueden inspeccionar diferentes tareas
de minería de texto:

Elegimos question answering y lo probamos con 2 modelos diferentes 

In [26]:
from transformers import pipeline

# Cargar el pipeline de QA con el modelo especificado
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Contexto y pregunta
contexto = """
El Amazonas es el río más largo del mundo, ubicado en América del Sur. Su longitud supera los 7,000 kilómetros.
"""
pregunta = "¿Dónde está ubicado el Amazonas?"

# Realizar la pregunta
respuesta = qa_pipeline(question=pregunta, context=contexto)

# Mostrar la respuesta
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']}")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pregunta: ¿Dónde está ubicado el Amazonas?
Respuesta: el río más largo del mundo, ubicado en América del Sur


In [11]:
from transformers import pipeline

# Cargar pipeline con otro modelo en español
qa_pipeline = pipeline(
    "question-answering",
    model="PlanTL-GOB-ES/roberta-base-bne-sqac",
    tokenizer="PlanTL-GOB-ES/roberta-base-bne-sqac",
    framework="pt"
)

contexto = "El Amazonas es el río más largo del mundo, ubicado en América del Sur."
pregunta = "¿Dónde está ubicado el Amazonas?"

respuesta = qa_pipeline(question=pregunta, context=contexto)
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']}")


Pregunta: ¿Dónde está ubicado el Amazonas?
Respuesta: América del Sur


4. Evaluar sobre el dataset elegido y hacer una comparativa de los modelos.

In [ ]:
%pip install transformers datasets evaluate

In [28]:
import pandas as pd
from evaluate import load
from datasets import Dataset
from transformers import pipeline

# Cargar las métricas
metric_squad = load("squad_v2")  # Usar "squad" si no se espera respuesta nula
metric_bleu = load("bleu")  # Métrica BLEU

# Datos de ejemplo
data = Dataset.from_dict({
    "context": [
        "El Amazonas es el río más largo del mundo, ubicado en América del Sur."
    ],
    "question": [
        "¿Dónde está ubicado el Amazonas?"
    ],
    "answers": [
        {"text": ["América del Sur"], "answer_start": [54]}
    ]
})

# Definir modelos a comparar
models = {
    "PlanTL-GOB-ES/roberta-base-bne-sqac": pipeline("question-answering", model="PlanTL-GOB-ES/roberta-base-bne-sqac"),
    "bert-large-uncased-whole-word-masking-finetuned-squad": pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad"),
}

# Crear lista para almacenar métricas de cada modelo
metrics_data = []

# Evaluación de los modelos
for model_name, qa_pipeline in models.items():
    predictions = []
    references = []

    for example in data:
        # Realizar la predicción
        prediction = qa_pipeline(
            question=example["question"],
            context=example["context"]
        )["answer"]

        # Agregar predicciones y referencias en el formato esperado
        predictions.append({
            "id": str(example["question"][0]),  # Usando la pregunta como ID
            "prediction_text": prediction,
            "no_answer_probability": 0.0  # Ajustar si es una pregunta sin respuesta
        })
        references.append({
            "id": str(example["question"][0]),
            "answers": [{"text": example["answers"]["text"][0], "answer_start": example["answers"]["answer_start"][0]}]
        })

    # Calcular las métricas SQUAD (F1 Score)
    squad_results = metric_squad.compute(predictions=predictions, references=references)
    f1_score = squad_results["f1"]

    # Calcular BLEU score
    bleu_results = metric_bleu.compute(predictions=[pred["prediction_text"] for pred in predictions], references=[[ref["answers"][0]["text"]] for ref in references])
    bleu_score = bleu_results["bleu"]

    # Almacenar las métricas en la lista
    metrics_data.append({
        "Modelo": model_name,
        "F1 Score": f1_score,
        "BLEU Score": bleu_score
    })

# Crear un DataFrame de pandas para mostrar la tabla
df_metrics = pd.DataFrame(metrics_data)

# Mostrar la tabla de métricas
print(df_metrics)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                              Modelo    F1 Score  BLEU Score
0                PlanTL-GOB-ES/roberta-base-bne-sqac  100.000000         0.0
1  bert-large-uncased-whole-word-masking-finetune...   42.857143         0.0


In [23]:
%pip install gradio

Defaulting to user installation because normal site-packages is not writeable
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
   ---------------------------------------- 0.0/57.2 MB ? eta -:--:--
   - -------------------------------------- 2.4/57.2 MB 12.2 MB/s eta 0:00:05
   --- ------------------------------------ 4.7/57.2 MB 11.9 MB/s eta 0:00:05
   ---- ----------------------------------- 6.8/57.2 MB 11.3 MB/s eta 0:00:05
   ------ --------------------------------- 9.2/57.2 MB 11.4 MB/s eta 0:00:05
   -------- ------------------------------- 11.5/57.2 MB 11.5 MB/s eta 0:00:04
   --------- ------------------------------ 14.2/57.2 MB 11.5 MB/s eta 0:00:04
   ----------- ---------------------------- 15.7/57.2 MB 11.6 MB/s eta 0:00:04
   ------------- -------------------------- 18.9/57.2 MB 11.7 MB/s eta 0:00:04
   -------------- ------------------------- 21.2/57.2 MB 11.7 MB/s eta 0:00:04
   ---------------- ----------------------- 23.6/57.2 MB 11.7 MB/s eta 0:00:03

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.0.0 requires fastapi<0.89.0, but you have fastapi 0.115.6 which is incompatible.


In [ ]:
import gradio as gr
from transformers import pipeline

# Cargar el pipeline de QA con el modelo elegido
qa_pipeline = pipeline(
    "question-answering",
    model="PlanTL-GOB-ES/roberta-base-bne-sqac",
    tokenizer="PlanTL-GOB-ES/roberta-base-bne-sqac",
    framework="pt"
)

# Función para realizar la predicción
def responder_pregunta(contexto, pregunta):
    respuesta = qa_pipeline(question=pregunta, context=contexto)
    return respuesta['answer']

# Crear la interfaz de Gradio
interface = gr.Interface(
    fn=responder_pregunta, 
    inputs=[
        gr.Textbox(label="Contexto", placeholder="Introduce el contexto aquí...", lines=5),
        gr.Textbox(label="Pregunta", placeholder="Introduce la pregunta aquí...")
    ], 
    outputs=gr.Textbox(label="Respuesta"),
    live=True
)

# Iniciar la interfaz
interface.launch(share=True)